In [1]:
# Standard modules
import itertools
from collections import OrderedDict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.time import Time
import progressbar


# Custom modules
from jpm_time_conversions import *
from jpm_logger import JpmLogger
from closest import closest
from jpm_number_printing import latex_float
from get_goes_flare_events import get_goes_flare_events
from determine_preflare_irradiance import determine_preflare_irradiance

__author__ = 'James Paul Mason'
__contact__ = 'jmason86@gmail.com'


def generate_jedi_catalog(threshold_time_prior_flare_minutes=240.0, 
                          dimming_window_relative_to_flare_minutes_left=0.0, 
                          dimming_window_relative_to_flare_minutes_right=240.0,
                          output_path='./', verbose=False):
    """Wrapper code for creating James's Extreme Ultraviolet Variability Experiment (EVE) Dimming Index (JEDI) catalog.

    Inputs:
        None.
        
    Optional Inputs:
        threshold_time_prior_flare_minutes [float]:             How long before a particular event does the last one need to have
                                                                occurred to be considered independent. If the previous one was too
                                                                recent, will use that event's pre-flare irradiance.
                                                                Default is 240 (4 hours).
        dimming_window_relative_to_flare_minutes_left [float]:  Defines the left side of the time window to search for dimming
                                                                relative to the GOES/XRS flare peak. Negative numbers mean 
                                                                minutes prior to the flare peak. Default is 0.0.
        dimming_window_relative_to_flare_minutes_right [float]: Defines the right side of the time window to search for dimming
                                                                relative to the GOES/XRS flare peak. If another flare
                                                                occurs before this, that time will define the end of the
                                                                window instead. Default is 240 (4 hours).
        threshold_minimum_dimming_window_minutes [float]:       The smallest allowed time window in which to search for dimming.
                                                                Default is 120.
        output_path [str]:                                      Set to a path for saving the JEDI catalog table and processing 
                                                                summary plots. Default is './', the current directory.
        verbose [bool]:                                         Set to log the processing messages to disk and console. Default is False.

    Outputs:
        No direct return, but writes a (csv? sql table? hdf5?) to disk with the dimming paramerization results.
        Subroutines also optionally save processing plots to disk in output_path.
                                                 
    Optional Outputs:
        None

    Example:
        generate_jedi_catalog(output_path='/Users/jmason86/Dropbox/Research/Postdoc_NASA/Analysis/Coronal Dimming Analysis/JEDI Catalog/',
                              verbose=True)
    """
    return 1

In [2]:
# prototype only: set up example inputs
%matplotlib inline
plt.style.use('jpm-transparent-light')

threshold_time_prior_flare_minutes = 240.0
dimming_window_relative_to_flare_minutes_left = 0.0 
dimming_window_relative_to_flare_minutes_right = 240.0
threshold_minimum_dimming_window_minutes = 120.0
output_path='/Users/jmason86/Dropbox/Research/Postdoc_NASA/Analysis/Coronal Dimming Analysis/JEDI Catalog/'
verbose = True

In [3]:
# Prepare the logger for verbose
if verbose:
    logger = JpmLogger(filename='generate_jedi_catalog', path=output_path)
    logger.info("Starting JEDI processing pipeline.")

2018-01-30 14:29:58,807 [INFO ] [<module>] Starting JEDI processing pipeline.


In [4]:
# Get EVE level 2 extracted emission lines data
# TODO: Replace this shortcut method with the method I'm building into sunpy
from scipy.io.idl import readsav
eve_readsav = readsav('/Users/jmason86/Dropbox/Research/Data/EVE/eve_lines_2010121-2014146 MEGS-A Mission Bare Bones.sav')

In [5]:
# prototype only: create metadata dictionary
# TODO: Replace this shortcut method with the method I'm building into sunpy
from sunpy.util.metadata import MetaDict
metadata = MetaDict()
metadata['ion'] = eve_readsav['name']
metadata['temperature_ion_peak_formation'] = np.power(10.0, eve_readsav['logt']) * u.Kelvin
metadata['extracted_wavelength_center'] = eve_readsav['wavelength'] * u.nm
metadata['extracted_wavelength_min'] = metadata['extracted_wavelength_center']
metadata['extracted_wavelength_max'] = metadata['extracted_wavelength_center']
metadata['emission_line_blends'] = ['none', 'yay', 'poop', 'Fe vi']  # etc
metadata['exposure_time'] = 60.0 * u.second  # These example EVE data are already binned down to 1 minute
metadata['precision'] = ['Not implemented in prototype']
metadata['accuracy'] = ['Not implemented in prototype']
metadata['flags'] = ['Not implemented in prototype']
metadata['flags_description'] = "1 = MEGS-A data is missing, 2 = MEGS-B data is missing, 4 = ESP data is missing, 8 = MEGS-P data is missing, 16 = Possible clock adjust in MEGS-A, 32 = Possible clock adjust in MEGS-B, 64 = Possible clock adjust in ESP, 128 = Possible clock adjust in MEGS-P"
metadata['flags_spacecraft'] = ['Not implemented in prototype']
metadata['flags_spacecraft_description'] = "0 = No obstruction, 1 = Warmup from Earth eclipse, 2 = Obstruction atmosphere penumbra, 3 = Obstruction atmosphere umbra, 4 = Obstruction penumbra of Mercury, 5 = Obstruction penumbra of Mercury, 6 = Obstruction penumbra of Venus, 7 = Obstruction umbra of Venus, 8 = Obstruction penumbra of Moon, 9 = Obstruction umbra of Moon, 10 = Obstruction penumbra of solid Earth, 11 = Obstruction umbra of solid Earth, 16 = Observatory is off-pointed by more than 1 arcmin"
metadata['data_version'] = ['Not implemented in prototype']
metadata['data_reprocessed_revision'] = ['Not implemented in prototype']
metadata['filename'] = ['Not implemented in prototype']

In [6]:
# prototype only: load up the actual irradiance data into a pandas DataFrame
# TODO: Replace this shortcut method with the method I'm building into sunpy
irradiance = eve_readsav['irradiance'].byteswap().newbyteorder()  # pandas doesn't like big endian
irradiance[irradiance == -1] = np.nan
wavelengths = eve_readsav['wavelength']
wavelengths_str = []
[wavelengths_str.append('{0:1.1f}'.format(wavelength)) for wavelength in wavelengths]
eve_lines = pd.DataFrame(irradiance, columns=wavelengths_str)
eve_lines.index = yyyydoy_sod_to_datetime(eve_readsav.yyyydoy, eve_readsav.sod)  # Convert EVE standard time to datetime

In [7]:
# Get GOES flare events above C1 within date range corresponding to EVE data
#flares = get_goes_flare_events(eve_lines.index[0], eve_lines.index[-1], verbose=verbose)  # TODO: The method in sunpy needs fixing, issue 2434

In [8]:
# prototype only: load GOES events from IDL saveset instead of directly through sunpy
goes_flare_events = readsav('/Users/jmason86/Dropbox/Research/Data/GOES/events/GoesEventsC1MinMegsAEra.sav')
goes_flare_events['class'] = goes_flare_events['class'].astype(str)
goes_flare_events['event_peak_time_human'] = goes_flare_events['event_peak_time_human'].astype(str)
goes_flare_events['event_start_time_human'] = goes_flare_events['event_start_time_human'].astype(str)
goes_flare_events['peak_time'] = Time(goes_flare_events['event_peak_time_jd'], format='jd', scale='utc')
goes_flare_events['start_time'] = Time(goes_flare_events['event_start_time_jd'], format='jd', scale='utc')
num_flares = len(goes_flare_events['class'])

In [19]:
# Define the columns of the JEDI catalog
jedi_row = pd.DataFrame([OrderedDict([("GOES Flare Start Time", np.nan),
                         ("GOES Flare Peak Time", np.nan),
                         ("GOES Flare Class", np.nan), 
                         ("Pre-Flare Start Time", np.nan),
                         ("Pre-Flare End Time", np.nan)])])
jedi_row = jedi_row.join(pd.DataFrame(columns=eve_lines.columns + ' Pre-Flare Irradiance [W/m2]'))
jedi_row = jedi_row.join(pd.DataFrame(columns=eve_lines.columns + ' Slope Start Time'))
jedi_row = jedi_row.join(pd.DataFrame(columns=eve_lines.columns + ' Slope End Time'))
jedi_row = jedi_row.join(pd.DataFrame(columns=eve_lines.columns + ' Slope [%/s]'))
jedi_row = jedi_row.join(pd.DataFrame(columns=eve_lines.columns + ' Slope Uncertainty [%/s]'))
jedi_row = jedi_row.join(pd.DataFrame(columns=eve_lines.columns + ' Depth Time'))
jedi_row = jedi_row.join(pd.DataFrame(columns=eve_lines.columns + ' Depth [%]'))
jedi_row = jedi_row.join(pd.DataFrame(columns=eve_lines.columns + ' Depth Uncertainty [%]'))
jedi_row = jedi_row.join(pd.DataFrame(columns=eve_lines.columns + ' Duration Start Time'))
jedi_row = jedi_row.join(pd.DataFrame(columns=eve_lines.columns + ' Duration End Time'))
jedi_row = jedi_row.join(pd.DataFrame(columns=eve_lines.columns + ' Duration [s]'))
jedi_row = jedi_row.join(pd.DataFrame(columns=eve_lines.columns + ' Flare Interrupt'))

ion_permutations = list(itertools.permutations(eve_lines.columns.values, 2))
ion_permutations = pd.Index([' by '.join(ion_permutations[i]) for i in range(len(ion_permutations))])

jedi_row = jedi_row.join(pd.DataFrame(columns=ion_permutations + ' Slope Start Time'))
jedi_row = jedi_row.join(pd.DataFrame(columns=ion_permutations + ' Slope End Time'))
jedi_row = jedi_row.join(pd.DataFrame(columns=ion_permutations + ' Slope [%/s]'))
jedi_row = jedi_row.join(pd.DataFrame(columns=ion_permutations + ' Slope Uncertainty [%/s]'))
jedi_row = jedi_row.join(pd.DataFrame(columns=ion_permutations + ' Depth Time'))
jedi_row = jedi_row.join(pd.DataFrame(columns=ion_permutations + ' Depth [%]'))
jedi_row = jedi_row.join(pd.DataFrame(columns=ion_permutations + ' Depth Uncertainty [%]'))
jedi_row = jedi_row.join(pd.DataFrame(columns=ion_permutations + ' Duration Start Time'))
jedi_row = jedi_row.join(pd.DataFrame(columns=ion_permutations + ' Duration End Time'))
jedi_row = jedi_row.join(pd.DataFrame(columns=ion_permutations + ' Duration [s]'))

csv_filename = output_path + 'jedi_{0}.csv'.format(Time.now().iso)
jedi_row.to_csv(csv_filename, header=True, mode='w')

In [10]:
# Start a progress bar
widgets = [progressbar.Percentage(), progressbar.Bar(), progressbar.Timer(), ' ', progressbar.AdaptiveETA()]
progress_bar = progressbar.ProgressBar(widgets=widgets, max_value=num_flares).start()
# just call progress_bar.update(i) to update this at later times

N/A%|                                     |Elapsed Time: 0:00:00 ETA:  --:--:--

In [11]:
# prototype only: import time just for example progress bar below
import time 

In [12]:
# Prepare a hold-over pre-flare irradiance value,
# which will normally have one element for each of the 39 emission lines
preflare_irradiance = None

# Start loop through all flares
for flare_index in range(num_flares):
    # all subsequent cells actually go here
    progress_bar.update(flare_index + 1)
    time.sleep(0.0002)

 97%|##################################### |Elapsed Time: 0:00:01 ETA:  0:00:00

In [13]:
# prototype only: reset flare_index to 1 (not 0) 
flare_index = 1

In [14]:
# Reset the flare interrupt flag
flare_interrupt = False

In [25]:
# Fill the GOES flare information into the JEDI row
jedi_row['GOES Flare Start Time'] = goes_flare_events['start_time'][flare_index].iso
jedi_row['GOES Flare Peak Time'] = goes_flare_events['peak_time'][flare_index].iso
jedi_row['GOES Flare Class'] = goes_flare_events['class'][flare_index]

In [27]:
# Determine pre-flare irradiance
minutes_since_last_flare = (goes_flare_events['peak_time'][flare_index] - goes_flare_events['peak_time'][flare_index - 1]).sec / 60.0
if  minutes_since_last_flare > threshold_time_prior_flare_minutes:
    # Clip EVE data from threshold_time_prior_flare_minutes prior to flare up to peak flare time
    preflare_window_start = (goes_flare_events['peak_time'][flare_index] - (threshold_time_prior_flare_minutes * u.minute)).iso
    preflare_window_end = (goes_flare_events['peak_time'][flare_index]).iso
    eve_lines_preflare_time = eve_lines[preflare_window_start:preflare_window_end]
    
    # Loop through the emission lines and get pre-flare irradiance for each
    preflare_irradiance = []
    for column in eve_lines_preflare_time:
        eve_line_preflare_time = pd.DataFrame(eve_lines_preflare_time[column])
        eve_line_preflare_time.columns = ['irradiance']
        preflare_irradiance.append(determine_preflare_irradiance(eve_line_preflare_time,
                                                                 pd.Timestamp(goes_flare_events['start_time'][flare_index].iso),
                                                                 verbose=verbose))
else: 
    logger.info("This flare at {0} will use the pre-flare irradiance from flare at {1}."
                .format(goes_flare_events['peak_time'][flare_index].iso, 
                        goes_flare_events['peak_time'][flare_index - 1].iso))

jedi_row["Pre-Flare Start Time"] = preflare_window_start
jedi_row["Pre-Flare End Time"] = preflare_window_end
preflare_irradiance_cols = [col for col in jedi_row.columns if 'Pre-Flare Irradiance' in col]
jedi_row[preflare_irradiance_cols] = preflare_irradiance

2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16

2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16:15:00.
2018-01-30 14:44:24,298 [INFO ] [determine_preflare_irradiance] Running on event with peak start time of 2010-05-04 16

2018-01-30 14:44:24,366 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-30 14:44:24,366 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-30 14:44:24,366 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-30 14:44:24,366 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-30 14:44:24,366 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-30 14:44:24,366 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.
2018-01-30 14:44:24,366 [INFO ] [determine_preflare_irradiance] Converted irradiance to percent, baselining median in entire pre-flare window.

2018-01-30 14:44:24,435 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:24,435 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:24,435 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:24,435 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:24,435 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:24,435 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:24,435 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:24,435 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:24,435 [INFO ] [determine_preflare_irra

2018-01-30 14:44:24,494 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-30 14:44:24,494 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-30 14:44:24,494 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-30 14:44:24,494 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-30 14:44:24,494 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-30 14:44:24,494 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-30 14:44:24,494 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-30 14:44:24,494 [INFO ] [determine_preflare_irradiance] Computed medians and standard deviations in each window.
2018-01-30 14:44:24,494 [INFO ] 

...

2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irra

2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irradiance] Divided pre-flare period into 3 equal-length windows.
2018-01-30 14:44:40,111 [INFO ] [determine_preflare_irra

2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more th

2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more th

2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more than 1 window is larger than threshold (0.5).
2018-01-30 14:44:40,310 [WARNI] [determine_preflare_irradiance] Cannot compute pre-flare irradiance. Standard deviation in more th

In [33]:
# Clip EVE data to dimming window
bracket_time_left = (goes_flare_events['peak_time'][flare_index] - (dimming_window_relative_to_flare_minutes_left * u.minute))
next_flare_time = (goes_flare_events['peak_time'][flare_index + 1]).iso
user_choice_time = (goes_flare_events['peak_time'][flare_index] + (dimming_window_relative_to_flare_minutes_right * u.minute))
bracket_time_right = min(next_flare_time, user_choice_time)

In [34]:
# If flare is shortening the window, set the flare_interrupt flag
if bracket_time_right == next_flare_time:
    flare_interrupt = 1
    if verbose:
        logger.info('Flare interrupt for event at {0} by flare at {1}'.format(goes_flare_events['peak_time'][flare_index].iso, next_flare_time))

In [39]:
# Skip event if the dimming window is too short
if ((bracket_time_right - bracket_time_left).sec / 60.0) < threshold_minimum_dimming_window_minutes:
    # Leave all dimming parameters as NaN and write this null result to the CSV on disk
    jedi_row.to_csv(csv_filename, header=False, mode = 'a')
    
    # Log message
    if verbose:
        logger.info('The dimming window duration of {0} minutes is shorter than the minimum threshold of {1} minutes. Skipping this event ({2})'
                    .format(((bracket_time_right - bracket_time_left).sec / 60.0), threshold_minimum_dimming_window_minutes, goes_flare_events['peak_time'][flare_index]))
    
    # Skip the rest of the processing in the flare_index loop
    # continue  # TODO: uncomment outside prototype
else:
    eve_lines_event = eve_lines[bracket_time_left.iso:bracket_time_right.iso]

In [43]:
# Convert irradiance units to percent